In [1]:
# Подклчаем нужные библиотеки
import json
import xlrd

In [2]:
# создаем функцию для преобразования .xlsx файла в .json
def ConvertXlsxToJson(xlsx_file, json_file):
    book = xlrd.open_workbook(xlsx_file)
    sh1 = book.sheet_by_index(0)
    store = []
    for rx in range(1, sh1.nrows):
        frame = {
            "global_id": int(sh1.row(rx)[0].value),
            "ID": sh1.row(rx)[1].value,
            "AdmArea": sh1.row(rx)[2].value,
            "District":sh1.row(rx)[3].value,
            "DescriptionLocation": sh1.row(rx)[4].value,
            "PayWay": sh1.row(rx)[5].value,
            "IntercityConnectionPayment": sh1.row(rx)[6].value,
            "ValidUniversalServicesCard": sh1.row(rx)[7].value,
            "signature_date" : sh1.row(rx)[8].value
        }
        store.append(frame)
    with open(json_file, 'w') as out_file:
        json_obj = json.dumps(store, ensure_ascii = False, indent = 4)
        out_file.write(json_obj)

In [3]:
#В .xlsx файл добавил атрибут для описания даты(формат даты не схож с датой из .json => надо привести к одному формату)
def resolve_format(date) -> str:
    if date == '':
        return '01.01.1970 00:00:00' # Если отсутсвует дата вообще, вставляет дефолтную дату
    date = date.split()
    if len(date) == 1:
        only_time = '00:00:00'
        only_date = date[0]
    else:
        only_date, only_time = date[0], date[1]
    only_date = only_date.split('.')
    if len(only_date[0]) > 2:
        only_date = only_date[1] + '.' + only_date[2] + '.' + only_date[0]
    else:
        only_date = only_date[0] + '.' + only_date[1] + '.' + only_date[2]
    return only_date + ' ' + only_time


# Создаем целевую схему для файлов .json
def scheme(file1, file2):
    store = []
    with open(file1, 'r', ) as f:
        dataMos = json.load(f)
    with open(file2, 'r', ) as f:
        dataGov = json.load(f)
    target_scheme = dataGov[0].keys()
    # Убирём два атрибуты, чтобы уменьшить количество столбцов и введем один новый вместо трех существующих
    target_scheme = set(target_scheme).symmetric_difference(set(["Longitude_WGS84",'AdmArea', \
                                                                 'District', 'DescriptionLocation', \
                                                                 'Address', "system_object_id"]))
    print(target_scheme)
    create_json(dataMos, file1, target_scheme)
    create_json(dataGov, file2, target_scheme)
    
    
# Создаем новые файлы .json с целевой схемой    
def create_json(data, json_file, target_scheme):
    store = []
    for obj in data:
        n_obj = dict()
        for attribute in target_scheme:
            if attribute == 'Address':
                # Три атрибута объединяем в один и будем в дальнейшем использовать как ключ(точное месторасположение)
                n_obj[attribute] = obj['AdmArea'] + ', ' + obj['District'] + ', ' + obj['DescriptionLocation']
            elif attribute not in obj.keys():
                n_obj[attribute] = None                
            elif attribute == 'signature_date':
                n_obj[attribute] = resolve_format(obj['signature_date'])
            else:
                n_obj[attribute] = obj[attribute]
        store.append(n_obj)
    with open('new_' + json_file, 'w') as out_file:
        json_obj = json.dumps(store, ensure_ascii = False, indent = 4)
        out_file.write(json_obj)    

In [4]:
ConvertXlsxToJson(xlsx_file = 'dataMos.xlsx', json_file = 'dataMos.json')
scheme('dataMos.json', 'dataGov.json')

{'Name', 'PayWay', 'global_id', 'signature_date', 'Address', 'Latitude_WGS84', 'ValidUniversalServicesCard', 'IntercityConnectionPayment', 'ID'}
